In [1]:
a = open("Articles.txt",'r')
b = a.read()
articles=[]
text=""
title=""
titles=[]
for i in b.split("-------------------"):
    if i =="":
        continue
    article = i.split('\n\n\n\n\n')
    text = article[1].strip('\n').strip(' ')
    if text.count('\n') <5:
        continue
    if text.count('https://') >10:
        continue   
    title = article[0].strip('\n').strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([title,text])
a.close()

In [4]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}
articleslen=[]
for i in articles:
    title,text = i[0],i[1]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [7]:
print(len(wordRepArticles))
# wordRepArticles[0]
print(len(articles))

3387
3387


In [6]:
articles[:5]

[["Senegal gets Africa's first NBA training center",
  'Lagos (CNN) The National Basketball Association (NBA) on Monday commissioned a new training center in Saly, Senegal, the first of its kind in Africa, as part of its drive to attract young teenagers to the league.\n\nThe new facility, which features two indoor basketball courts, a multidimensional activity center, a weight room, conference rooms as well as dormitories and educational facilities, will be the primary training location for NBA Academy Africa future hopefuls from across the continent, the league said in a statement.\n\n"We are thrilled to open this new world-class venue in Saly for NBA Academy Africa prospects," said Amadou Gallo Fall, vice president and managing director of NBA Africa.\n\n"This facility will provide aspiring players from across the continent with state-of-the-art courts and training equipment, under the tutelage of our NBA coaches and our world-class academic staff," he said.\n\nThe NBA Academy Africa

In [8]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(len(articles)):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,len(articles)):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [9]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][0])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][0])

Main title: Senegal gets Africa's first NBA training center
0/3: Meet Neku Atawodi: The woman powering tech investments in Africa
1/3: LA, Dallas, NY-NJ , DC Headline XFL Cities for First Season
2/3: NHL Greenlights Seattle Expansion Franchise
Main title: Police arrest 6 after a teenage boy died in an adulthood initiation ritual in Kenya
0/3: California teacher arrested for forcibly cutting boy's hair
1/3: Quiz of the Week: What's this boy's claim to fame?
2/3: Sister of attacked teenage Syrian refugee also targeted by bullies in UK school
Main title: At least 35 people are dead and dozens are missing after a boat accident in Uganda
0/3: Shock Video: Migrants Land Boat On California Beach 100 Miles From Border
1/3: Woman killed on train may have been leaning out of window, say police
2/3: Woman killed 'leaning out of train window' between Bath and Bristol
Main title: US Embassy in DRC warns of "possible terrorist threat" against US facilities
0/3: Ebola cases surge in Congo, with women

Main title: British boxer sends message of support to bullied Syrian refugee
0/3: Sister of attacked teenage Syrian refugee also targeted by bullies in UK school
1/3: UK student to be charged after Syrian refugee attacked
2/3: Tommy Robinson threatened with legal action over 'bully' video
Main title: Escaped snow leopard shot and killed in zoo
0/3: With 1 bullet, a police officer is shot in the face and a suspect is shot dead during a struggle: Officials
1/3: Massive lion fends off more than 20 hyenas during attack, wild video shows
2/3: CDC warns about disease threat from Asian longhorned tick
Main title: Left to go it alone, Ukraine's navy waits for Russia's next move
0/3: U.S. carries out "extraordinary" flight over Ukraine, citing Russian attack
1/3: Ukraine bans Russian men aged 16 to 60 from entry
2/3: In interview, Ukrainian president asks Trump to deliver pointed message to Putin
Main title: After 5 months at sea, Ross Edgley completes swim around Great Britain
0/3: Ross Edgley

Main title: Revealed: the stark evidence of everyday racial bias in Britain
0/3: This is a vital study of racial bias. Now will Britain take heed? | Afua Hirsch
1/3: 'They think I'm a gangster': the young black men caught in joblessness
2/3: Poll Shows Huge Democrat Bias Toward Muslims Over Christians
Main title: This is a vital study of racial bias. Now will Britain take heed? | Afua Hirsch
0/3: Revealed: the stark evidence of everyday racial bias in Britain
1/3: Crowdsourced: What's the right way to tell someone you don't like their holiday gift?
2/3: 'They think I'm a gangster': the young black men caught in joblessness
Main title: Theresa May in battle over Brexit legal advice as charm offensive begins
0/3: UK faces constitutional crisis over Brexit legal advice, Labour warns
1/3: Brexit legal advice warns UK could be trapped in endless negotiations
2/3: UK's Labour will seek no confidence vote in May if Brexit bill fails
Main title: Paris riots: PM to meet protest groups after wor

Main title: NHS hospitals treat soaring number of older people for drug misuse
0/3: Medical advances could soon spare patients surgery, say experts
1/3: One in five patients now waits at least two weeks to see a GP, finds report
2/3: Sharp rise in number of young people seeking help for anxiety
Main title: Killing of swan family in Kent brings calls for airgun regulation
0/3: With 1 bullet, a police officer is shot in the face and a suspect is shot dead during a struggle: Officials
1/3: Iran suicide bomber kills 2 outside police station, state media reports
2/3: Woman killed on train may have been leaning out of window, say police
Main title: AI and drones turn an eye towards UK's energy infrastructure
0/3: Facebook exec says the social network would be 'dust' without AI
1/3: Reality Check: Your Christmas tree's carbon footprint
2/3: Start-up that uses A.I. to detect phone fraudsters with more than 99% accuracy raises $90 million
Main title: Met police numbers 'will fall dramatically w

Main title: Send a tip on an independent UK music venue for a chance to win a £200 hotel voucher
0/3: The best small UK music venues: readers’ travel tips
1/3: Your pictures: share your photos on the theme of 'wait'
2/3: Share your photos of the worst Christmas trees
Main title: Teenage boys and young men: tell us what issues matter to you
0/3: Were you inspired by Extinction Rebellion to become an activist?
1/3: Your pictures: share your photos on the theme of 'wait'
2/3: Quiz of the Week: What's this boy's claim to fame?
Main title: A new council in London’s East End: a bid for democracy or island of privilege?
0/3: Developers may have to enhance wildlife habitats, says Gove
1/3: Pelosi: I Won’t Support Wall Funding in Exchange for DACA Deal
2/3: London's 'Tulip' tower proposal worries air traffic control
Main title: Do we need to hide who we are to speak freely in the era of identity politics?
0/3: Why Is This Happening? Assessing America's information crisis with David Roberts: pod

...

1/3: Remembering the Life of Former President George H.W. Bush
2/3: Former President George H.W. Bush's funeral in Texas: Live stream
Main title: Judge Andrew Napolitano: Why I don't believe that Mueller is on a fishing expedition (or is about to go home)
0/3: Mueller probe: Prosecutors given Friday deadline for sentencing memo on former Trump lawyer Michael Cohen
1/3: Read special counsel Mueller's sentencing memo that recommends no jail time for Michael Flynn
2/3: Mueller Expected To Reveal Details About 3 Major Players In Russia Investigation
Main title: Laura Ingraham: Democrats are now the smartest people in the room even if you're too stupid to understand that
0/3: Ingraham: Some in Media Reduce Bush's Death to 'Political Battering Ram' Against Trump
1/3: Maureen Dowd: George H.W. Bush Called Trump a Curse Word, Threw His Shoe at TV When He Appeared
2/3: The New 'Deplorables' Insult? 'The Five' Responds to HI Sen. Hirono Boasting About Dems' Intellect
Main title: Leslie Marshall:

0/3: UC Berkeley Settles Lawsuit over Conservative Speaker Censorship
1/3: The Todd Starnes Show – December 4th, 2018
2/3: Former Executive Assistant to Ronald Reagan Peggy Grande, Spokesperson for YAF Spencer Brown & Novelist Brad Thor
Main title: A Mauve Colored Brine
0/3: The Special Relationship
1/3: It Makes Me Sick Just Thinking About It
2/3: I’ll Tell You What…Hail to the Chief
Main title: The Special Relationship
0/3: A Mauve Colored Brine
1/3: It Makes Me Sick Just Thinking About It
2/3: I’ll Tell You What…Hail to the Chief
Main title: It Makes Me Sick Just Thinking About It
0/3: A Mauve Colored Brine
1/3: The Special Relationship
2/3: I’ll Tell You What…Hail to the Chief
Main title: Down the Midterm Stretch We Go
0/3: Fit4Service
1/3: A Front Row Seat Inside the Presidency with Lara Trump
2/3: A Mauve Colored Brine
Main title: A Front Row Seat Inside the Presidency with Lara Trump
0/3: Fit4Service
1/3: Down the Midterm Stretch We Go
2/3: Neil Cavuto Premium Podcast
Main title

0/3: Iran suicide bomber kills 2 outside police station, state media reports
1/3: Breaking: Migrant Caravan Set To Storm US Border Saturday Due To Mexico Eviction
2/3: Egyptian actress could face 5 years in prison for wearing revealing dress
Main title: Family of U.S. missionary John Chau: We forgive tribe for killing him
0/3: North Sentinel Island and the strange death of John Allen Chau – podcast
1/3: Three arrested in connection with police killing during 'cow vigilante' clashes in India
2/3: This Week in Pictures
Main title: No-deal Brexit would cost Britain 'tens of billions,' finance minister warns
0/3: Brexit deal is worse than staying in E.U., lawmaker who helped draft it says
1/3: Brexit will put focus on 'talents and skills,' not E.U. passports, May says
2/3: Third of Theresa May's Trade Envoys Will Not Back Her Brexit Deal
Main title: Russia loses Interpol presidential bid after U.S. lobbying campaign
0/3: Pompeo's talks with North Korea are postponed without explanation
1/3

Main title: Pakistan pivots toward Putin's Russia after Trump gets tough
0/3: Jim Mattis to Pakistan: Time to End War in Afghanistan
1/3: Trump Urges Pakistan PM to Assist U.S. on Taliban Peace Deal
2/3: Pakistan Welcomes Trump’s Approach to Ending Afghan War
Main title: Bittersweet reunions for North and South Korean families separated by war
0/3: Pompeo's talks with North Korea are postponed without explanation
1/3: S. Korea's Moon: Trump wants to grant Kim Jong Un's wishes
2/3: Ban Ki-moon urges N. Korea to take denuclearization steps
Main title: Officials worry Trump may back Erik Prince plan to privatize war in Afghanistan
0/3: Secretary of State Mike Pompeo’s Bid to Senators to Stick with Saudi Arabia
1/3: Jim Mattis to Pakistan: Time to End War in Afghanistan
2/3: Afghan Taliban claims indirect talks are underway with U.S.
Main title: 'Go Back to Where You Came From' retraces reverse refugee route
0/3: Help Syrians survive the conflict
1/3: Aquarius ordeal highlights hardening v

Main title: David A. Andelman As Trump alienates allies at home and abroad, Pence's Asia trip highlights his competent diplomatic skills
0/3: Shares advance in Asia following Xi-Trump tariffs truce
1/3: Trump says he believes China's Xi Jinping on trade tariff truce
2/3: Trump Closes Major Trade Deal With Canada, Mexico
Main title: Holiday detoxes promise to cleanse our bodies of toxins — but this wellness trend sells more snake oil than science
0/3: One-month sugar detox: A nutritionist explains how and why
1/3: Military diet: 3-day diet or dud?
2/3: These foods may help keep the brain young
Main title: Good teachers help kids find themselves represented in the history of America
0/3: 2 former refugees make history with midterm victories
1/3: How the Louvre Abu Dhabi became a place of discovery for students
2/3: The essential Ta-Nehisi Coates
Main title: James Ball The Brexit debate is currently just the deluded fighting with those in denial. The U.K. is in trouble.
0/3: Brexit vote l

In [10]:
print(avgArticleLen)

766.0442869796279
